In [1]:
import numpy as np
import itertools
from numba import jit, prange

In [2]:
# parameters of the simulation
n_encodings=4
n_ammino=5
n_h=3
n_h_val=2

In [3]:
# needs to be replaced by actual simulation values
w=np.zeros((20,3)) #weights matrix
v=np.zeros(20) #visible layer
a=np.zeros(20) #visible bias
b=np.zeros(3) #hidden bias
h=np.zeros(3) #hidden layer

Definition of energy
$$E=-\sum_{i,j} v_i w_{i,j} h_j-\sum_i a_i v_i-\sum_j b_j h_j$$

Definition of the single probability
$$p(v,h|a,b,w)=e^{-E(v,h|a,b,w)}/Z$$

Definition of partition function
$$Z=\sum_{\{v,h\}} e^{-E}$$

Definition of LogLikelihoof function but badly written
$$\mathcal{L}=M^{-1}\sum_{i}^M ln\big[p(v=v_i^{(M)}|a,b,w)\big]$$

LogLikelihood but easier to compute
$$\mathcal{L}=M^{-1}\left(\sum_i^M{ln\left[\sum_{\{h\}}e^{-E[v_i,h|a,b,w]}\right]}\right) -ln Z$$

In [4]:
def E(v, h, w, a, b):
    '''
    computes the energy of the configuration. 
    depends on v, h, a, b, w
    '''
    return np.dot(v, np.dot(w, h))+np.dot(a,v)+np.dot(b,h)

In [5]:
def P(v_data, possible_h, w, a, b):
    '''
    returns the first contribution to the loglikelihood second form
    '''
    p=0
    for v_i in v_data:
        sum_h=0
        for h_j in possible_h:
            sum_h+=np.exp(E(v_i, h_j, w, a, b))
        p+=np.log(sum_h)
    return p/v_data.shape[0]

In [6]:
def Z(possible_v, possible_h, w, a, b):
    '''
    just the partition function
    depends on w: has to be computed at each epoch
    because of changes in w calues
    '''
    z=0
    for v_i in possible_v:
        for h_j in possible_h:
            z+=np.exp(E(v_i, h_j, w, a, b))
    return z

In [7]:
def Log(w, a, b):
    '''
    summing of contributions
    '''
    return P(v_data, possible_h, w, a, b)-np.log(Z(possible_v, possible_h, w, a, b))

In [8]:
def Log_2(w,h_current,a,b):
    '''
    New definition of Log Likelihood
    needs the current state of hidden variables as input
    returns the current value of the loglikelihood
    avg_en is the simple arithmetic energy for each epoch
    eq. 189 of mehta for reference
    '''
    avg_en=0
    for v in v_data:
        avg_en+=E(v, h_current, w, a, b )
    return -avg_en/v_data.shape[0]-np.log(Z(possible_v, possible_h, w, a, b))

In [9]:
# this code should be run only once
# possibile_* has to be available to all the functions as global variables
# possibile_* never change

#list all possible combinations of proteins
possibilities_v=list(itertools.product(['1000','0100','0010','0001'], repeat=n_ammino))
possibilities_h=list(itertools.product(['1','0'], repeat=n_h))

possible_v=np.zeros((n_encodings**n_ammino, n_encodings*n_ammino))
possible_h=np.zeros((n_h_val**n_h, n_h))

#converts proteins to decimal representation
for i,v in enumerate(possibilities_v):
    possible_v[i]=np.array([int(i) for i in ''.join(v)])

for i,h in enumerate(possibilities_h):
    possible_h[i]=np.array([int(i) for i in ''.join(h)])

In [10]:
# use the real dataset
v_data=np.random.randn(10000,20)

In [11]:
# a stupid test
Log(w, a, b)

-6.931471805599594